### Tutorial Notebook: G<sub>0</sub>W<sub>0</sub> Approximation 

In this notebook, we present an example calculation of quasiparticle energies using QuatumMASALA's `gw` module.

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Imports
import numpy as np
import sys

sys.path.append(".")

dirname = "./"

In [3]:
import numpy as np

from qtm.constants import RYDBERG, ELECTRONVOLT
from qtm.kpts import KList
from qtm.lattice import RealLattice
from qtm.crystal import BasisAtoms, Crystal
from qtm.pseudo import UPFv2Data
# from qtm.klist import gen_monkhorst_pack_grid
from qtm.gspace import GSpace
from qtm.mpi import QTMComm
from qtm.dft import DFTCommMod, scf

from qtm.io_utils.dft_printers import print_scf_status

from qtm import qtmconfig
from qtm.logger import qtmlogger
# qtmconfig.fft_backend = 'mkl_fft'

from mpi4py.MPI import COMM_WORLD
comm_world = QTMComm(COMM_WORLD)
dftcomm = DFTCommMod(comm_world)

# Lattice
reallat = RealLattice.from_alat(alat=10.2,  # Bohr
                                a1=[-0.5,  0.0,  0.5],
                                a2=[ 0.0,  0.5,  0.5],
                                a3=[-0.5,  0.5,  0.0])

# Atom Basis
si_oncv = UPFv2Data.from_file('Si_ONCV_PBE-1.2.upf')
si_atoms = BasisAtoms.from_alat('Si', si_oncv, 28.086, reallat,
                               [[0.875, 0.875, 0.875], [0.125, 0.125, 0.125]])

crystal = Crystal(reallat, [si_atoms, ])  # Represents the crystal


# Generating k-points from a Monkhorst Pack grid (reduced to the crystal's IBZ)
mpgrid_shape = (4, 4, 4)
x = np.linspace(0,1,mpgrid_shape[0], endpoint=False)
y = np.linspace(0,1,mpgrid_shape[1], endpoint=False)
z = np.linspace(0,1,mpgrid_shape[2], endpoint=False)
xx,yy,zz = np.meshgrid(x,y,z, indexing="ij")
kcryst = np.vstack([xx.flatten(),yy.flatten(),zz.flatten()])

kpts = KList(recilat=crystal.recilat, k_coords=kcryst, k_weights=np.ones(kcryst.shape[1])/kcryst.shape[1])

# mpgrid_shift = (True,True,True)
# kpts = gen_monkhorst_pack_grid(crystal, mpgrid_shape, mpgrid_shift)
# print(kpts.numkpts)
# -----Setting up G-Space of calculation-----
ecut_wfn = 25 * RYDBERG
# NOTE: In future version, hard grid (charge/pot) and smooth-grid (wavefun)
# can be set independently
ecut_rho = 4 * ecut_wfn
grho = GSpace(crystal.recilat, ecut_rho)
gwfn = grho

# -----Spin-polarized (collinear) calculation-----
is_spin, is_noncolin = False, False
# Starting with asymmetric spin distribution else convergence may yield only
# non-magnetized states
mag_start = [0.0]
numbnd = 30  # Ensure adequate # of bands if system is not an insulator

occ = 'fixed'
smear_typ = 'gauss'
e_temp = 0.0001 * RYDBERG

conv_thr = 1E-8 * RYDBERG
diago_thr_init = 1E-2 * RYDBERG

In [4]:
from qtm.constants import ELECTRONVOLT_HART
from qtm.kpts import KList


kpts = KList(recilat=crystal.recilat, k_coords=kpts.k_cryst, k_weights=np.ones(kpts.k_cryst.shape[1])/kpts.k_cryst.shape[1])

out = scf(dftcomm, crystal, kpts, grho, gwfn,
          numbnd, is_spin, is_noncolin,
          rho_start=mag_start, occ_typ=occ,
          conv_thr=conv_thr, diago_thr_init=diago_thr_init,
          iter_printer=print_scf_status,
          ret_vxc=True)

scf_converged, rho, l_wfn_kgrp, en, vxc = out


print("SCF Routine has exited")
print(qtmlogger)

# print(vxc/ELECTRONVOLT_HART)

# for wfn in l_wfn_kgrp:
#     wfn.evc_gk/=(np.linalg.norm(wfn.evc_gk, axis=-1, keepdims=True))
    # wfn.evl*=2
    # wfn.normalize()

Iteration # 1, Run Time:   8.5 sec
Convergence Status   : NOT Converged
SCF Error           : 6.1171e-02 Ry
Avg Diago Iterations: 7.7
Diago Threshold     : 1.00e-02 Ry

Total Energy:          -15.96754455 Ry

      one-el:            4.83802701 Ry
     Hartree:            0.80642513 Ry
          XC:           -4.71223800 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            5.89767081 eV
    LU Level:            6.86611554 eV
----------------------------------------

Iteration # 1, Run Time:  11.7 sec
Convergence Status   : NOT Converged
SCF Error           : 6.1010e-02 Ry
Avg Diago Iterations: 2.6
Diago Threshold     : 7.63e-04 Ry

Total Energy:          -15.96792935 Ry

      one-el:            4.83764221 Ry
     Hartree:            0.80642513 Ry
          XC:           -4.71223800 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            5.89675663 eV
    LU Level:            6.86552231 eV
----------------------------------------

Iteration # 2, Run Time:  14

/home/agrimsharma/codes/QuantumMASALA_demo/QuantumMASALA/src/qtm/dft/scf.py:438: ComplexWarning: Casting complex values to real discards the imaginary part
  vxc_arr[ik,iband] = hpsi_r


SCF Routine has exited
                          TIMERS                           
-----------------------------------------------------------
|            LABEL             |  CALL  |  TIME  | STATUS  |
-----------------------------------------------------------
-----------------------------------------------------------

                COUNTERS                 
-----------------------------------------
|            LABEL             | COUNT  |
-----------------------------------------
-----------------------------------------



In [5]:
kpts_q = KList(recilat=crystal.recilat, k_coords=kpts.k_cryst+np.array([[0,0,0.001]]).T, k_weights=np.ones(kpts.k_cryst.shape[1])/kpts.k_cryst.shape[0])

out_q = scf(dftcomm, crystal, kpts_q, grho, gwfn,
          numbnd, is_spin, is_noncolin,
          rho_start=mag_start, occ_typ=occ, smear_typ='gauss', e_temp=e_temp,
          conv_thr=conv_thr, diago_thr_init=diago_thr_init,
          iter_printer=print_scf_status)

scf_converged_q, rho_q, l_wfn_kgrp_q, en_q = out_q


print("Shifted SCF Routine has exited")
print(qtmlogger)

Iteration # 1, Run Time:  10.1 sec
Convergence Status   : NOT Converged
SCF Error           : 6.1181e-02 Ry
Avg Diago Iterations: 7.7
Diago Threshold     : 1.00e-02 Ry

Total Energy:           -5.33847822 Ry

      one-el:           15.46709334 Ry
     Hartree:            0.80642513 Ry
          XC:           -4.71223800 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            5.89781265 eV
    LU Level:            6.86455466 eV
----------------------------------------

Iteration # 1, Run Time:  13.7 sec
Convergence Status   : NOT Converged
SCF Error           : 6.1007e-02 Ry
Avg Diago Iterations: 2.5
Diago Threshold     : 7.63e-04 Ry

Total Energy:           -5.34521954 Ry

      one-el:           15.46035202 Ry
     Hartree:            0.80642513 Ry
          XC:           -4.71223800 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            5.89673909 eV
    LU Level:            6.86372562 eV
----------------------------------------

Iteration # 2, Run Time:  17

### Load Input Files
Input data is handled by the ``EpsInp`` class.\
The data can be provided either by constructing the ``EpsInp`` object or by reading BGW-compatible input file ``epsilon.inp``.\
The attributes have been supplied with docstrings from BerkeleyGW's input specification, so they will be accessible directly in most IDEs.

### Load Input Files
Input data is handled by the ``EpsInp`` class.\
The data can be provided either by constructing the ``EpsInp`` object or by reading BGW-compatible input file ``epsilon.inp``.\
The attributes have been supplied with docstrings from BerkeleyGW's input specification, so they will be accessible directly in most IDEs.

In [6]:
from qtm.gw.io_bgw.epsinp import Epsinp

# Constructing input manually
# epsinp = Epsinp(epsilon_cutoff=1.2,
#                 use_wfn_hdf5=True,
#                 number_bands=8,
#                 write_vcoul=True,
#                 qpts=[[0.0,0.0,0.0]],
#                 is_q0=[True])

# Reading from epsilon.inp file
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')
# print(epsinp)

# There is an analogous system to read SigmaInp
from qtm.gw.io_bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')
# print(sigmainp)

### Initialize Epsilon Class

``Epsilon`` class can be initialized by either directly passing the required `quantummasala.core` objects or by passing the input objects discussed earlier.

In [7]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon
from qtm.klist import KList

kpts_gw =   KList(recilat=kpts.recilat,   cryst=kpts.k_cryst.T,   weights=kpts.k_weights)
kpts_gw_q = KList(recilat=kpts_q.recilat, cryst=kpts_q.k_cryst.T, weights=kpts_q.k_weights)

# Manual initialization
epsilon = Epsilon(
    crystal = crystal,
    gspace = grho,
    kpts = kpts_gw,
    kptsq = kpts_gw_q,        
    l_wfn = l_wfn_kgrp,
    l_wfnq = l_wfn_kgrp_q,
    qpts = QPoints.from_cryst(kpts.recilat, epsinp.is_q0, *epsinp.qpts),
    epsinp = epsinp,
)

# epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 11015.45it/s]


The three main steps involved in the calculation have been mapped to the corresponding functions:
1.  ``matrix_elements``: Calculation of Planewave Matrix elements
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
3.  ``epsilon_inverse``: Calculation of (static) epsilon-inverse matrix

<!-- 1.  ``matrix_elements``: Calculation of Planewave Matrix elements
    $$M_{nn'}({\textbf k},{\textbf q},{\textbf G}) = \bra{n\,{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n'\,\textbf k}$$
    where the $\textbf G$-vectors included in the calculation satisfy $|\textbf q + \textbf G|^2 < E_{\text{cut}}$.
    Since this is a convolution in k-space, the time complexity can be reduced from $\mathcal{O}\left(N^2_G\right)$ to $\mathcal{O}\left(N_G\ln N_G\right)$ by using Fast Fourier Transform, where $N_G$  the number of reciprocal lattice vectors in the wavefunction.
    $$
    M_{nn'}({\bf k},{\bf q},\{{\bf G}\}) = {\rm FFT}^{-1}\left( \phi^{*}_{n,{\bf k}+{\bf q} }({\bf r}) \phi_{n',{\bf k} }({\bf r}) \right).
    $$
    where $\phi_{n',{\bf k}}({\bf r}) = {\rm FFT}\left( \psi_{n\bf k}(\bf G)\right)$. 
    
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
    $$
        P_{\textbf{GG'}}{\left({\textbf q}\;\!;0\right)}=
        \,\,{}\sum_{n}^{\textrm occ}\sum_{n'}^{\textrm emp}\sum_{{\textbf k}}
        \frac{
        \bra{n'\textbf k}e^{-i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n{\textbf k}{+}{\textbf q}}
        \bra{n{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G'})\cdot{\textbf r}}\ket{n'\textbf k}
        }{E_{n{\textbf k}{+}{\textbf q}}\,{-}\,E_{n'{\textbf k}}}.
    $$
3.  ``epsilon_inverse``: Calculation of (static) epsilon-inverse matrix
    $$
        \epsilon_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}=
        \delta_{\textbf{GG'}}\,{-}\,v{\left({\textbf q}{+}{\textbf G}\right)} \,
        P_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}
    $$
    where $ v(\textbf{q} + \textbf{G}) = \frac{8\pi}{\left|\textbf q + \textbf G\right|^2} $ is bare Coulomb potential, written in Rydberg units. If this formula is used as-is for the case where $|\textbf q| = |\textbf G| = 0$, the resulting $v\left({\textbf{q=0}, \textbf{G=0}}\;\!\right)$ blows up as $1/q^2$. However, for 3D gapped systems, the matrix elements $\big| M_{nn'}\left({\bf k},{\textbf{q}\to\textbf{0}},{\textbf{G=0}}\right)\big| \sim q$ cancel the Coulomb divergence and $\epsilon_{\textbf{00}}\left({\textbf q\to\textbf{0}}\;\!\right) \sim q^2/q^2$ which is a finite number. In order to calculate $\epsilon_{\textbf{00}}\left({\textbf q=\textbf{0}}\;\!\right)$, we use the scheme specified in BGW2012, wherein $q=0$ is replaced with a small but non-zero value. Since matrix element calculation involves the eigenvectors $\ket{n{\textbf k}{+}{\textbf q}}$, having a non-$\Gamma$-centered $\textbf q\to 0$ point requires mean-field eigenvectors over a shifted $k$-grid. -->

In [8]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))
        # if i_q==0:
        #     N=10
        #     print(M.shape)
        #     print(M[0,0,0,:N].real)
        #     break

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # chimat/=2
        
        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv

        if i_q == epsilon.qpts.index_q0:
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsmats.append(epsinv)
            
        if False:        
            # Compare the results with BGW's results
            if i_q == epsilon.qpts.index_q0:
                epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            else:
                epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])    

            # Calculate stddev between reference and calculated epsinv matrices
            std_eps = np.std(np.abs(epsref) - np.abs(epsinv)) / np.sqrt(np.prod(list(epsinv.shape)))
        
            epstol = 1e-16
            if np.abs(std_eps) > epstol:
                print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")
                print(np.max(np.abs(epsinv-epsref)))
                indices = np.where(np.abs(epsinv)-np.abs(epsref)>1e-5)
                

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)

calculate_epsilon()


Epsilon> q-pt index:   0%|          | 0/64 [00:00<?, ?it/s]

Epsilon> q-pt index: 100%|██████████| 64/64 [00:30<00:00,  2.10it/s]


### Sigma Calculation

In [9]:
from qtm.gw.sigma import Sigma

outdir = dirname+"temp/"


sigma = Sigma.from_qtm_scf(
    crystal = crystal,
    gspace = grho,
    kpts = kpts_gw,
    kptsq=kpts_gw_q,
    l_wfn_kgrp=l_wfn_kgrp,
    l_wfn_kgrp_q=l_wfn_kgrp_q,
    sigmainp=sigmainp,
    epsinp = epsinp,
    epsilon=epsilon,
    rho=rho,
    vxc=vxc
)

# sigma = Sigma.from_data(
#     wfndata=wfndata,
#     wfnqdata=wfnqdata,
#     sigmainp=sigmainp,
#     epsinp=epsinp,
#     l_epsmats=epsilon.l_epsinv,
#     rho=rho,
#     vxc=vxc,
#     outdir=outdir,
# )

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 9831.72it/s]


vcoul: Vcoul:
        * gspace = <qtm.gspace.gspc.GSpace object at 0x7fcea21de210>
        * qpts = <qtm.gw.core.QPoints object at 0x7fce7f185a10>
        * bare_coulomb_cutoff = 10.0
        * avgcut = 1e-05
        * l_gspace_q = <class 'list'> of length 64
        * vcoul = <class 'list'> of length 64
        * N_SAMPLES = 2500000.0
        * N_SAMPLES_COARSE = 250000.0
        * SEED = 5000
        


Vcoul calculation for qpts: 100%|██████████| 64/64 [01:45<00:00,  1.65s/it]


In [10]:
sigma_sx_cohsex_mat = sigma.sigma_sx_static(yielding=True)    
print("Sigma SX COHSEX")
sigma.pprint_sigma_mat(sigma_sx_cohsex_mat)

(137,)


Sigma_SX_Static:   3%|▎         | 2/64 [00:00<00:03, 17.73it/s]

Sigma_SX_Static: 100%|██████████| 64/64 [00:03<00:00, 18.54it/s]

Sigma SX COHSEX
   11.875836   10.668109   11.328765
    8.031077   10.682459   10.014678
    8.031134    8.719223    8.366075
    8.032073    8.720117    8.365870
    2.911686    2.587455    2.872630
    2.912302    2.587258    2.425368
    2.906995    1.329107    2.428446
    2.681682    1.328693    0.739609


In [11]:
sigma_ch_cohsex_mat = sigma.sigma_ch_static()    
print("Sigma CH COHSEX")
sigma.pprint_sigma_mat(sigma_ch_cohsex_mat)

Sigma_CH_Static_Partial:   2%|▏         | 1/64 [00:00<00:06,  9.24it/s]

Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:06<00:00, 10.61it/s]

Sigma CH COHSEX
   -6.627248   -6.303004   -6.494451
   -5.099187   -6.303227   -6.068627
   -5.099443   -5.571220   -5.407186
   -5.099480   -5.571186   -5.407365
   -4.746638   -4.701166   -4.917357
   -4.747086   -4.701184   -4.286824
   -4.744842   -3.460646   -4.287561
   -4.757445   -3.460008   -2.975094


In [12]:
sigma.autosave=False
sigma.print_condition=True
cohsex_result = sigma.calculate_static_cohsex()

Sigma_X:   3%|▎         | 2/64 [00:00<00:03, 18.62it/s]

Sigma_X: 100%|██████████| 64/64 [00:03<00:00, 19.65it/s]


Sigma X
  -17.549889  -16.038878  -16.918927
  -12.974870  -16.038756  -14.908715
  -12.974884  -13.417485  -13.204607
  -12.975035  -13.417626  -13.204309
   -5.765691   -5.196397   -5.934483
   -5.765496   -5.196268   -5.081733
   -5.765279   -3.789599   -5.081638
   -5.831193   -3.789627   -2.406334
(137,)


Sigma_SX_Static: 100%|██████████| 64/64 [00:03<00:00, 18.26it/s]


Sigma SX STATIC
   11.875836   10.668109   11.328765
    8.031077   10.682459   10.014678
    8.031134    8.719223    8.366075
    8.032073    8.720117    8.365870
    2.911686    2.587455    2.872630
    2.912302    2.587258    2.425368
    2.906995    1.329107    2.428446
    2.681682    1.328693    0.739609


Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:06<00:00, 10.21it/s]


Sigma CH STATIC
   -6.627248   -6.303004   -6.494451
   -5.099187   -6.303227   -6.068627
   -5.099443   -5.571220   -5.407186
   -5.099480   -5.571186   -5.407365
   -4.746638   -4.701166   -4.917357
   -4.747086   -4.701184   -4.286824
   -4.744842   -3.460646   -4.287561
   -4.757445   -3.460008   -2.975094


Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:39<00:00,  1.63it/s]

Sigma CH EXACT STATIC
   -7.444555   -7.623150   -7.626662
   -7.756878   -7.623196   -7.314288
   -7.756964   -7.476666   -7.673956
   -7.757280   -7.477583   -7.674273
   -7.335558   -6.795163   -7.354452
   -7.335933   -6.795007   -7.120203
   -7.334057   -7.674458   -7.120734
   -7.829751   -7.673536   -6.232123
Sig (Exact):
  -13.118609  -12.993919  -13.216824
  -12.700671  -12.979493  -12.208325
  -12.700714  -12.174928  -12.512488
  -12.700242  -12.175091  -12.512712
  -10.189563   -9.404105  -10.416305
  -10.189127   -9.404017   -9.776569
  -10.192341  -10.134951   -9.773926
  -10.979262  -10.134471   -7.898848
Eqp0 (Exact):
   -8.251844   -3.524031   -5.575222
    5.142299   -3.509604   -2.586362
    5.142256    2.014950    3.845950
    5.142728    2.014787    3.845726
    8.897982    6.723119    7.761276
    8.898417    6.723208    9.692298
    8.895204   17.163899    9.694941
    9.966476   17.164380   14.034580
Sig (Partial):
  -12.301302  -11.673773  -12.084614
  -10.04298

In [13]:
sigma.print_condition=True
sigma_ch_exact_mat = sigma.sigma_ch_static_exact()    
print("Sigma CH COHSEX EXACT")
sigma.pprint_sigma_mat(sigma_ch_exact_mat)

Sigma_CH_Static_Exact:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:35<00:00,  1.80it/s]

Sigma CH COHSEX EXACT
   -7.444555   -7.623150   -7.626662
   -7.756878   -7.623196   -7.314288
   -7.756964   -7.476666   -7.673956
   -7.757280   -7.477583   -7.674273
   -7.335558   -6.795163   -7.354452
   -7.335933   -6.795007   -7.120203
   -7.334057   -7.674458   -7.120734
   -7.829751   -7.673536   -6.232123


In [20]:
sigma.print_condition=False
sigma_sx_gpp = sigma.sigma_sx_gpp()    
print("Sigma SX GPP")
sigma.pprint_sigma_mat(sigma_sx_gpp)

Sigma_SX_GPP:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_SX_GPP: 100%|██████████| 64/64 [01:19<00:00,  1.24s/it]

Sigma SX GPP
  n     ik=0          ik=40         ik=8      
  1    12.244002   10.834603   11.549690
  2     8.177192   10.849576   10.174693
  3     8.178080    8.837332    8.501004
  4     8.178604    8.839068    8.501182
  5     3.298948    2.866268    3.299154
  6     3.299935    2.865697    2.788043
  7     3.289248    1.738717    2.793553
  8     3.371348    1.736905    1.089234


In [21]:
sigma.print_condition=False
sigma_ch_gpp,_ = sigma.sigma_ch_gpp()    
print("Sigma CH GPP")
sigma.pprint_sigma_mat(sigma_ch_gpp)

Sigma_CH_GPP: 100%|██████████| 64/64 [01:31<00:00,  1.42s/it]

Sigma CH GPP
  n     ik=0          ik=40         ik=8      
  1    -5.796970   -5.846048   -5.879221
  2    -5.402500   -5.849967   -5.642406
  3    -5.403558   -5.558926   -5.572885
  4    -5.403264   -5.559721   -5.573241
  5    -5.342389   -4.983154   -5.446887
  6    -5.343420   -4.982612   -4.888681
  7    -5.334747   -4.622626   -4.892131
  8    -5.723835   -4.621018   -3.771267


In [26]:
gpp_result = sigma.calculate_gpp()

Started calculate_gpp Rank: 0 2023-09-23 18:00:39


Sigma_X: 100%|██████████| 64/64 [00:03<00:00, 20.03it/s]

Sigma X GPP


  n      ik=1        ik=41       ik=9    
  1   -17.549889  -16.038878  -16.918927
  2   -12.974870  -16.038756  -14.908715
  3   -12.974884  -13.417485  -13.204607
  4   -12.975035  -13.417626  -13.204309
  5    -5.765691   -5.196397   -5.934483
  6    -5.765496   -5.196268   -5.081733
  7    -5.765279   -3.789599   -5.081638
  8    -5.831193   -3.789627   -2.406334


Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:05<00:00, 11.20it/s]

...

Sigma_CH_GPP: 100%|██████████| 64/64 [01:28<00:00,  1.38s/it]

Sigma CH GPP
  n      ik=1        ik=41       ik=9    
  1    -5.796970   -5.846048   -5.879221
  2    -5.402500   -5.849967   -5.642406
  3    -5.403558   -5.558926   -5.572885
  4    -5.403264   -5.559721   -5.573241
  5    -5.342389   -4.983154   -5.446887
  6    -5.343420   -4.982612   -4.888681
  7    -5.334747   -4.622626   -4.892131
  8    -5.723835   -4.621018   -3.771267
Sig GPP:
  n      ik=1        ik=41       ik=9    
  1   -11.511511  -11.710397  -11.814563
  2   -11.529023  -11.699132  -10.999258
  3   -11.529123  -11.091802  -11.409874
  4   -11.528595  -11.091477  -11.409823
  5    -9.103591   -8.360282   -9.300763
  6    -9.103403   -8.360095   -8.599062
  7    -9.105386   -8.780414   -8.596802
  8    -9.719833   -8.780504   -6.716880
Eqp0


  n      ik=1        ik=41       ik=9    
  1    -6.644746   -2.240509   -4.172961
  2     6.313947   -2.229243   -1.377295
  3     6.313847    3.098076    4.948564
  4     6.314374    3.098401    4.948615
  5     9.983953    7.766942    8.876818
  6     9.984141    7.767129   10.869804
  7     9.982158   18.518436   10.872065
  8    11.225904   18.518347   15.216548
dE [[0.07349864 0.07349864 0.07349864 0.07349864 0.07349864 0.07349864
  0.07349864 0.07349864]
 [0.07349864 0.07349864 0.07349864 0.07349864 0.07349864 0.07349864
  0.07349864 0.07349864]
 [0.07349864 0.07349864 0.07349864 0.07349864 0.07349864 0.07349864
  0.07349864 0.07349864]]


Sigma_SX_GPP: 100%|██████████| 64/64 [01:23<00:00,  1.30s/it]

Sigma CH GPP dE
  n      ik=1        ik=41       ik=9    
  1    -6.056252   -6.122846   -6.149626
  2    -5.640594   -6.127126   -5.916333
  3    -5.637499   -5.843145   -5.860695
  4    -5.639991   -5.843902   -5.861296
  5    -5.569557   -5.287112   -5.772181
  6    -5.570467   -5.286651   -5.097664
  7    -5.560974   -4.909714   -5.101438
  8    -5.940856   -4.907959   -4.062506
Sigma SX GPP dE
  n      ik=1        ik=41       ik=9    
  1    -6.056252   -6.122846   -6.149626
  2    -5.640594   -6.127126   -5.916333
  3    -5.637499   -5.843145   -5.860695
  4    -5.639991   -5.843902   -5.861296
  5    -5.569557   -5.287112   -5.772181
  6    -5.570467   -5.286651   -5.097664
  7    -5.560974   -4.909714   -5.101438
  8    -5.940856   -4.907959   -4.062506
Z:
  n      ik=1        ik=41       ik=9    
  1     0.714876    0.775182    0.755487
  2     0.839969    0.775101    0.778938
  3     0.839963    0.817122    0.829062
  4     0.839965    0.817159    0.829068
  5     0.830708   